In [1]:
import re
import math
import numpy as np

In [2]:
class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  def preprocessing(self, sequences):
    result = []
    '''
    문제 1-1.
    '''
    for sequence in sequences:
      temp_list = sequence.lower().split()
      fixed_temp_list = []
      for token in temp_list:
        fixed_token = re.sub(r"[^a-z]","",token)
        fixed_temp_list.append(fixed_token)
      result.append(fixed_temp_list)
    return result
  
  def fit(self, sequences):
    self.fit_checker = False
    '''
    문제 1-2.
    '''
    processed_list = self.preprocessing(sequences)
    token_num = 1
    for sentence in processed_list:
      for token in sentence:
        if token in self.word_dict: continue
        else: 
          self.word_dict[token] = token_num
          token_num +=1

    self.fit_checker = True
  
  def transform(self, sequences):
    result = []
    tokens = self.preprocessing(sequences)
    if self.fit_checker:
      '''
      문제 1-3.
      '''
      for sentence in tokens:
        sentence_idx_list = []
        for word in sentence:
          if self.word_dict[word]:sentence_idx_list.append(self.word_dict[word])
          else:sentence_idx_list.append(self.word_dict['oov'])
        result.append(sentence_idx_list) 
      return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result

In [4]:
class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False

  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)
    '''
    문제 2-1.
    '''
    result = []
    n = len(tokenized)
    token_num = max(self.tokenizer.word_dict.values())
    df_list=[]
    
    for i in range(1,token_num+1):
      df = 0
      for sentence in tokenized:
        if i in sentence: 
          df+=1
          continue
      df_list.append(df)
      idf = math.log(n/(1+df))
      result.append(idf)
    
    self.fit_checker = True
    return result
    

  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)
      '''
      문제 2-2.
      '''
      self.tfidf_matrix = []
      idf = self.fit(sequences)
      tf = []
      n = len(tokenized)
      token_list = list(self.tokenizer.word_dict.values())
      token_num = max(token_list)
      for sentence in tokenized:
        sentence_tf = [0]*token_num
        for token in sentence:
          sentence_tf[token-1]+=1 # 문장에 토큰이 있으면 갯수 up
        
        tf_idf_indiv = list(np.multiply(idf,sentence_tf))
        self.tfidf_matrix.append(tf_idf_indiv)
      return self.tfidf_matrix
    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")

  
  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)